In [ ]:
# Normal Working Flow OF RAG

In [2]:
##simple genai app using langchain
import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT="pr-shadowy-jazz-89"
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")

In [6]:
# step 1 from data ingestion ->scrap data from Web
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/observability/how_to_guides/threads")
docs=loader.load()
docs



[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nSet up threads | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesUse monitoring chartsDashboardsLog traces to specific projectSet up automation rulesOnline EvaluationSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog multimodal tracesLog retriever tracesLog custom LLM tracesPrevent logging of se

In [ ]:
for chunks in docs:
    print(chunks)

page_content='




Set up threads | 🦜️🛠️ LangSmith






Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesUse monitoring chartsDashboardsLog traces to specific projectSet up automation rulesOnline EvaluationSet a sampling rate for tracesAdd metadata and tags to tracesImplement distributed tracingAccess the current run (span) within a traced functionLog multimodal tracesLog retriever tracesLog custom LLM tracesPrevent logging of sensitive data in tracesQuery tracesShare or unshare a trace publiclyCompare tracesTrace generator functionsTrace with LangChain (Python and JS/TS)Trace with LangGraph (Python and JS/TS)Trace with Instructor (Python only)Trace with OpenTelemetryTrace with the Vercel AI SDK

In [20]:
###load data-->Docs--->divide into tunks--->text-->vectors-->vectorembedding-->vector
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
documents=text_splitter.split_documents(docs)
documents






[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Set up threads | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityQuick StartTutorialsAdd observability to your LLM applicationHow-to GuidesAnnotate code for tracingFilter traces in the applicationUpload files with tracesUse monitoring chartsDashboardsLog traces to specific'),
 Document(metadata

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
embeedings=HuggingFaceEmbeddings()


In [22]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeedings)
vectorstoredb

In [23]:
query="A Thread is a sequence of traces representing a single conversation"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Group traces into threads\u200b\nA Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.\nTo associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.'

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain_openai import ChatOpenAI-same as gemini

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


In [39]:
##retevial chain->document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """ 
      Answet the following question based only on the provided text:
      <context>
      {context}
      </context>
      Question: {input}
      Provide a detailed explanation in your answer.
    """
)
doc_chain=create_stuff_documents_chain(llm,prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template=' \n      Answet the following question based only on the provided text:\n      <context>\n      {context}\n      </context>\n      Question: {input}\n      Provide a detailed explanation in your answer.\n    '), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x769f08142030>, default_metadata=())
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factori

In [ ]:
##just a test part
from langchain_core.documents import Document
doc_chain.invoke(
    {
        "input":"A Thread is a sequence of traces representing a single conversation",
        "context":[Document(page_content="A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread")],
    }
)

'That is correct. The provided text states: "A Thread is a sequence of traces representing a single conversation."'

In [40]:
## input-->Reterver-->vector store databse
retirver=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
rt_chain=create_retrieval_chain(retirver,doc_chain)#context information get from doc_chain and VDB from retiever
rt_chain



RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x769f08140320>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template=' \n      Answet the following question based only on the provided text:\n      <context>\n      {context}\n      </context>\n     

In [41]:
##get the response from The LLM
response=rt_chain.invoke({"input":"A Thread is a sequence of traces representing a single conversation"})
response['answer']

'A Thread is defined as a sequence of traces. These traces are linked together because they are part of the same conversation. Each response in the conversation is represented as an individual trace within the thread.'

In [44]:
response

{'input': 'A Thread is a sequence of traces representing a single conversation',
 'context': [Document(id='e30f838b-c32e-4958-aec4-f5fd92e2abbf', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Group traces into threads\u200b\nA Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.\nTo associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.'),
  Document(id='73ddcee5-0222-4a38-a5dc-7b772cad13a7', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this conten

In [ ]:
response['context']#you will know the context has taken

[Document(id='e30f838b-c32e-4958-aec4-f5fd92e2abbf', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Group traces into threads\u200b\nA Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.\nTo associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.'),
 Document(id='73ddcee5-0222-4a38-a5dc-7b772cad13a7', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/threads', 'title': 'Set up threads | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='session_id\nt